<a href="https://colab.research.google.com/github/anujdubey1996/LLM---Hands-On/blob/main/LLM_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun May 19 19:43:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              51W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 18.3 M

In [3]:
!pip install xformers

In [4]:
!nvidia-smi

Sun May 19 19:46:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.4 MB/s eta 0:00:00


In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [9]:
from pathlib import Path

from huggingface_hub import notebook_login
from tqdm import tqdm

notebook_login()

# Prompt Generation (Text)

In [1]:
import random

def generate_prompt(task, num_prompts=1):
    """
    Generates prompts for different LLM tasks.

    Args:
        task (str): The type of task (e.g., "text_generation", "question_answering", "summarization", "translation").
        num_prompts (int, optional): The number of prompts to generate. Defaults to 1.

    Returns:
        list: A list of generated prompts.
    """
    prompts = []
    for _ in range(num_prompts):
        if task == "text_generation":
            settings = [
                "a haunted house on a hill",
                "a bustling city street",
                "a peaceful forest clearing"
            ]
            prompt = f"Write a short story about {random.choice(settings)}."
        elif task == "question_answering":
            topics = [
                "science",
                "history",
                "geography",
                "technology"
            ]
            prompt = f"What is the most important invention in the field of {random.choice(topics)}?"
        elif task == "summarization":
            sources = [
                "news article",
                "scientific paper",
                "short story"
            ]
            prompt = f"Summarize the following {random.choice(sources)} in one sentence:"
        elif task == "translation":
          languages = [
              ("English", "French"),
              ("Spanish", "German"),
              ("Japanese", "English")
          ]
          source_lang, target_lang = random.choice(languages)
          sentences = {
              "English": "The cat is on the mat.",
              "Spanish": "El perro está en la casa.",
              "Japanese": "猫はマットの上にいます。"
          }
          prompt = f"Translate the following sentence from {source_lang} to {target_lang}: {sentences[source_lang]}"
        else:
            raise ValueError("Invalid task type")

        prompts.append(prompt)

    return prompts


In [2]:
# Example Usage:
tasks = ["text_generation", "question_answering", "translation"]
initial_prompts = []
for task in tasks:
  print(task)
  initial_prompts.append(generate_prompt(task, num_prompts=10))

text_generation
question_answering
translation


In [3]:
initial_prompts

[['Write a short story about a peaceful forest clearing.',
  'Write a short story about a bustling city street.',
  'Write a short story about a peaceful forest clearing.',
  'Write a short story about a bustling city street.',
  'Write a short story about a bustling city street.',
  'Write a short story about a bustling city street.',
  'Write a short story about a peaceful forest clearing.',
  'Write a short story about a haunted house on a hill.',
  'Write a short story about a peaceful forest clearing.',
  'Write a short story about a haunted house on a hill.'],
 ['What is the most important invention in the field of science?',
  'What is the most important invention in the field of science?',
  'What is the most important invention in the field of science?',
  'What is the most important invention in the field of history?',
  'What is the most important invention in the field of geography?',
  'What is the most important invention in the field of history?',
  'What is the most imp

In [4]:
import torch
from transformers import PreTrainedTokenizer, pipeline

text_pipeline = pipeline(
    "text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float32,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def prompt_template(text, tokenizer):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful and harmless AI assistant",
        },
        {"role": "user", "content": text},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )


In [6]:
prompts = []
for prompt_task in initial_prompts:
  add = []
  for prompt in prompt_task:
    #print(prompt)
    add.append(prompt_template(prompt, text_pipeline.tokenizer))
    #print(add)
  prompts.append(add)

In [7]:
import os
from pathlib import Path

folder_path = Path("prompts")
folder_path.mkdir(parents=True, exist_ok=True)

os.chdir(folder_path)

for j, task_prompt in enumerate(prompts, start=1):
  folder_path = Path(f"task_{j}")
  folder_path.mkdir(parents=True, exist_ok=True)
  for i, prompt in enumerate(task_prompt, start=1):
      file_path = folder_path / f"prompt_{i}.txt"
      file_path.write_text(prompt)

# Hugging Face Model

In [ ]:
prompts

['<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a haunted house on a hill. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a peaceful forest clearing. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a peaceful forest clearing. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a bustling city street. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a haunted house on a hill. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a haunted house on a hill. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a peaceful forest clearing. [/INST]',
 '<s>[INST] <

In [8]:
!nvidia-smi

Sun May 19 20:00:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              57W / 400W |  26269MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import time
from datasets import Dataset

start_time = time.time()

# Set the pad_token_id to the eos_token_id (end of sequence token)
text_pipeline.tokenizer.pad_token_id = text_pipeline.tokenizer.eos_token_id
dataset = Dataset.from_dict({"text": prompts})
outputs = text_pipeline(dataset["text"], batch_size=10)
end_time = time.time()

print(end_time-start_time)

133.30771732330322


In [ ]:
import gc
torch.cuda.empty_cache()
del text_pipeline
gc.collect()

NameError: name 'text_pipeline' is not defined

# vLLM

In [1]:
import torch
from vllm import LLM, SamplingParams

In [2]:
llm = LLM(model="meta-llama/Llama-2-7b-chat-hf", dtype=torch.float32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 05-19 20:01:15 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='meta-llama/Llama-2-7b-chat-hf', speculative_config=None, tokenizer='meta-llama/Llama-2-7b-chat-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Llama-2-7b-chat-hf)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 05-19 20:01:15 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-19 20:01:16 selector.py:74] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 05-19 20:01:16 selector.py:32] Using XFormers backend.
INFO 05-19 20:01:18 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 05-19 20:01:30 model_runner.py:175] Loading model weights took 25.1045 GB
INFO 05-19 20:01:34 gpu_executor.py:114] # GPU blocks: 595, # CPU blocks: 256
INFO 05-19 20:01:36 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-19 20:01:36 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the 

In [3]:
def prompt_template(text, tokenizer):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful and harmless AI assistant",
        },
        {"role": "user", "content": text},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [5]:
import os
from pathlib import Path

parent_folder="prompts"
all_prompts = []  # Store prompts as a list of lists
base_path = Path(parent_folder)

# Iterate over numbered subfolders
for task_folder in sorted(base_path.iterdir(), key=lambda p: p.name):
    if not task_folder.is_dir():
        continue

    prompts_for_task = []
    # Iterate over .txt files in the subfolder
    for prompt_file in task_folder.glob("*.txt"):
        with open(prompt_file, "r") as f:
            prompts_for_task.append(f.read().strip())

    all_prompts.append(prompts_for_task)

In [7]:
prompts = [prompt for task in all_prompts for prompt in task]
prompts

['<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a peaceful forest clearing. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a bustling city street. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a peaceful forest clearing. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a haunted house on a hill. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a peaceful forest clearing. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a bustling city street. [/INST]',
 '<s>[INST] <<SYS>>\nYou are a helpful and harmless AI assistant\n<</SYS>>\n\nWrite a short story about a bustling city street. [/INST]',
 '<s>[INST] <<SYS>>

In [9]:
import time
from datasets import Dataset

start_time = time.time()

dataset = Dataset.from_dict({"text": prompts})
sampling_params = SamplingParams(temperature=0.1, max_tokens=256)
outputs = llm.generate(prompts, sampling_params)

end_time = time.time()

print(end_time-start_time)

Processed prompts: 100%|██████████| 30/30 [00:11<00:00,  2.60it/s]

11.59559178352356


In [10]:
print(end_time-start_time)

11.59559178352356
